# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# This function help us to checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)
    

#### Create Keyspace

In [7]:
#Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## New table and insert values to Query 1 question:
**Description:** In this CQL statement you can find two components from Primary Key:  sessionId is the partition key (which determine the distribution of the data) and itemInSession is clustering key (will sort the data ir sorted ascending). This query search the information on the music database with session Id and Item-session.

In [9]:
song_playlist_session_id_CQL = "CREATE TABLE IF NOT EXISTS song_playlist_session_id"
song_playlist_session_id_CQL = song_playlist_session_id_CQL + "(artist text, song_title text, length_song float, sessionId int, item_number int, PRIMARY KEY(sessionId, item_number))"            

In [10]:
try:
    session.execute(song_playlist_session_id_CQL)
except Exception as e:
    print(e)

In [11]:
#The next step will be assign the csv file to data ingestion:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Now, We assign the INSERT statements into the `song_playlist_session_id` variable
        query = "INSERT INTO song_playlist_session_id(artist, song_title, length_song, sessionId, item_number)"
        query = query + "VALUES(%s, %s, %s, %s, %s);"
        ## We must assign the corresponding column element for each column in the INSERT statement.
        session.execute(query,(line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### The next step, is a control validation to verify that the data have been inserted into the table song_playlist_session_id. We limit the search until five records.

In [12]:
query_control = "SELECT artist, song_title FROM song_playlist_session_id limit 5"
try:
    validation_data = session.execute(query_control)
except Exception as e:
    print(e)

In [13]:
def pandas_factory(colnames, rows):
    """
    Return a dataframe throught pandas to get a best visualization
    
    Args:
        colnames: Headers from the info
        rows: The records that compose to the dataframe
    """
    return pd.DataFrame(rows, columns=colnames)

In [14]:
session.row_factory = pandas_factory
df = validation_data._current_rows
print(df)

[Row(artist='Regina Spektor', song_title='The Calculation (Album Version)'), Row(artist='Octopus Project', song_title='All Of The Champs That Ever Lived'), Row(artist='Tegan And Sara', song_title='So Jealous'), Row(artist='Dragonette', song_title='Okay Dolores'), Row(artist='Lil Wayne / Eminem', song_title='Drop The World')]


### Solution exercise 1

In [15]:
## The next CQL statement help us to get the artist name, song title and length song according to item and session id:
result_exercise_1 = session.execute("""
    SELECT artist, song_title, length_song
    FROM song_playlist_session_id
    WHERE
        item_number=4 AND
        sessionid=338;
""")
df = result_exercise_1._current_rows 
df

,artist,song_title,length_song
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## New table and insert values to Query 2 question:
**Description:** In this CQL statement you can find two components from Primary Key: userId is the partition key (which determine the distribution of the data) and sessionId is clustering key (will sort the data ir sorted ascending). This query search the information on the music database with user Id and session id.

In [16]:
## The next CQL statement help us to get the artist name, song title, and the information associated
## according to item and session id:
song_playlist_user_id_CQL = "CREATE TABLE IF NOT EXISTS songs_playlist_userId"
song_playlist_user_id_CQL = song_playlist_user_id_CQL + "(artist text, song_title text, userId int, firstName text, lastName text, sessionId int, item_number int, PRIMARY KEY((userId, sessionId), item_number))"


In [17]:
try:
    session.execute(song_playlist_user_id_CQL)
except Exception as e:
    print(e)

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Now, We assign the INSERT statements into the `song_playlist_user_id` variable
        query = "INSERT INTO songs_playlist_userId(artist, song_title, userId, firstName, lastName, sessionId, item_number)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s);"
          ## We must assign the corresponding column element for each column in the INSERT statement.
          ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[0], line[9], int(line[10]), line[1], line[4], int(line[8]), int(line[3])))

In [19]:
## The next CQL statement help us to get the artist name, song title and user id according to item and session id:

result_exercise_2 = session.execute("""
            SELECT artist, song_title, userId
            FROM songs_playlist_userId
            WHERE sessionId = 182
            AND userId = 10
            ORDER BY item_number
            """) 
df2 = result_exercise_2._current_rows 
df2

,artist,song_title,userid
0,Down To The Bone,Keep On Keepin' On,10
1,Three Drives,Greece 2000,10
2,Sebastien Tellier,Kilometer,10
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


## New table and insert values to Query 3 question:
**Description:** In this CQL statement you can find two components from Primary Key: song_title is the partition key (which determine the distribution of the data), then,  first name and last name is clustering key (will sort the data ir sorted ascending). This query search the information on the music database with this three components.

In [22]:
## The next CQL statement help us to get the song title, first and last name from the user according to the primary key define below:
##3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
song_playlist_user_info_CQL = "CREATE TABLE IF NOT EXISTS songs_playlist_userInfo"
song_playlist_user_info_CQL = song_playlist_user_info_CQL + "(song_title text, firstName text, lastName text, userId int, PRIMARY KEY(song_title, userId))"            

In [23]:
try:
    session.execute(song_playlist_user_info_CQL)
except Exception as e:
    print(e)

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Now, We assign the INSERT statements into the `songs_playlist_userInfo` variable
        query = "INSERT INTO songs_playlist_userInfo(song_title, firstName, lastName, userId)"
        query = query + "VALUES(%s, %s, %s, %s);"
        ## We must assign the corresponding column element for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[9], line[1], line[4], int(line[10])))

In [25]:
result_exercise_3 = session.execute("""
            SELECT firstName, lastName
            FROM songs_playlist_userInfo
            WHERE song_title = 'All Hands Against His Own'
            """) 
df3 = result_exercise_3._current_rows 
df3

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions
*Description:* Finally, We'll Drop the tables before closing out the session:


In [1]:
try:
    session.execute("DROP TABLE song_playlist_session_id")
    session.execute("DROP TABLE songs_playlist_userId")
    session.execute("DROP TABLE songs_playlist_userInfo")
except Exception as e:
    print(e)

name 'session' is not defined


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()